# Tensorflow で定義される Estimator を使った SageMaker の学習と推論

#### ノートブックに含まれる内容

- Tensorflow の `tf.estimator` を SageMaker で使うときの基本的なやりかた
- `input_fn` を使った，推論時の入力データに対する前処理

#### ノートブックで使われている手法の詳細

- アルゴリズム: `tf.estimator.DNNClassifier`
- データ: iris

## セットアップ

必要なパラメタをセットします．

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## データのロード

この例では，SageMaker の S3 バケットにすでに用意されているデータをそのまま使うので，特にデータロードの必要はありません．

[Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set) には，150 行のデータが含まれており，3 種類のアヤメ（Iris setosa, Iris versicolor, Iris virginica）のデータがそれぞれ 50 行ずつ存在します．

各行のデータには，がく片の長さと幅，花びらの長さと幅が含まれています．最後のカラムは，[0|1|2] に符号化された，アヤメの種類です．このチュートリアルでは，150 行のデータのうち 120 行を `iris_training.csv` として学習用に，30 行を `iris_test.csv` として評価用に使います．

Sepal Length | Sepal Width | Petal Length | Petal Width | Species
:----------- | :---------- | :----------- | :---------- | :-------
5.1          | 3.5         | 1.4          | 0.2         | 0
4.9          | 3.0         | 1.4          | 0.2         | 0
4.7          | 3.2         | 1.3          | 0.2         | 0
&hellip;     | &hellip;    | &hellip;     | &hellip;    | &hellip;
7.0          | 3.2         | 4.7          | 1.4         | 1
6.4          | 3.2         | 4.5          | 1.5         | 1
6.9          | 3.1         | 4.9          | 1.5         | 1
&hellip;     | &hellip;    | &hellip;     | &hellip;    | &hellip;
6.5          | 3.0         | 5.2          | 2.0         | 2
6.2          | 3.4         | 5.4          | 2.3         | 2
5.9          | 3.0         | 5.1          | 1.8         | 2

以下を実行する前に，**<span style="color: red;">`data/iris/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/iris/XX')

## Tensorflow で `tf.estimator` および  `input_fn` を使うときのスクリプトの中身を確認

### `tf.estimator`

tensorflow にはあらかじめ定番の Estimator が用意されているため，Sagemaker でもこれを利用することができます．この Estimator の詳細については，[こちら](https://www.tensorflow.org/extend/estimators) から確認することができます．

- `tf.estimator.LinearClassifier`: 線形分類モデル
- `tf.estimator.LinearRegressor`: 線形回帰モデル
- `tf.estimator.DNNClassifier`: ディープニューラルネットワーク分類モデル
- `tf.estimator.DNNRegressor`: ディープニューラルネットワーク回帰モデル

通常はモデルを定義するために `model_fn` を用いますが，既存の `tf.estimator` を使用する場合には，`model_fn` の代わりに `estimator_fn` を用います．今回は定番の `tf.estimator.DNNClassifier` を用いるため，`estimator_fn` を使用します．

### `input_fn`

推論時のリクエストは，以下のようなフローとして処理されます．`input_fn` および `output_fn` はオプショナルなメソッドで，特に前処理および後処理を行う必要がない場合には，記述する必要はありません．詳細については[こちら](https://github.com/aws/sagemaker-python-sdk#model-serving)をご覧ください．

```python
# invokeEndpoint を叩いたリクエストの中身が，まず input_fn に送られますので，ここで必要な前処理を行います
input_object = input_fn(request_body, request_content_type)

# input_fn の出力をもとに，デプロイしたモデルで予測を行い，結果を返します
# このメソッドはオーバーライドすることはできません．SageMaker 側で自動的に処理が行われます
prediction = predict_fn(input_object, model)

# 予測結果がクライアントに返される前に，output_fn で後処理を行います
ouput = output_fn(prediction, response_content_type)
```

今回は，pickle 形式でリクエスト Body を受け取って，これを Tensorflow Serving に引渡し可能な配列形式に変換します．以下のコマンドを叩いて，実際にスクリプトの中身を確認してみてください．

In [ ]:
!cat "iris_dnn_classifier.py"

## モデルの学習を実行

学習時の記述は，通常の Tensorflow の実行と変わりはありません．

In [ ]:
from sagemaker.tensorflow import TensorFlow

iris_estimator = TensorFlow(entry_point='iris_dnn_classifier.py',
                            role=role,
                            train_instance_count=1,
                            train_instance_type='ml.m4.xlarge',
                            training_steps=100,
                            evaluation_steps=10)

iris_estimator.fit(inputs)

## モデルの推論を実行

`deploy()` メソッドを使って，学習済みモデルのデプロイを実施します．それが完了したら，`predict()` メソッドで実際に予測を行ってみます．

In [ ]:
%%time
iris_predictor = iris_estimator.deploy(initial_instance_count=1,
                                       instance_type='ml.m4.xlarge')

`input_fn` では，pickle 形式でリクエスト Body を受け取って，これを Tensorflow Serving に引渡し可能な配列形式に変換します．ですのでリクエストデータを pickle 形式に変換してからリクエストを投げます（`estimator.predict()` では `ContentType` の設定ができないため，ここでは boto3 ライブラリから API を直接叩いています）．

In [ ]:
import numpy as np
import pickle
import boto3

client = boto3.client('sagemaker-runtime')
pickled = pickle.dumps([3.4, 2.2, 1.5, 6.5])
response = client.invoke_endpoint(
    EndpointName=iris_predictor.endpoint,
    Body=pickled,
    ContentType='application/python-pickle')
print(response["Body"].read())


## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
import sagemaker

sagemaker_session.delete_endpoint(iris_predictor.endpoint)